# Operational Examples

## Show dblk size
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/byte`

## Show dblk current record number
`ls -al /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.recnum`

## Show dblk current last known record offset
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.last_rec`

## Show dblk current last known SYNC record offset
`ls -l /<node_id:ffffffffffff>/tag/sd/0/img/dblk/.last_sync`

## `tail -f` dblk file
`tail -f /<node_id:ffffffffffff>/tag/sd/0/img/dblk/byte`

In [1]:
from __future__ import print_function, absolute_import, division

import logging
import os
import sys
from collections import defaultdict, OrderedDict
from errno import ENOENT, ENODATA
from stat import S_IFDIR, S_IFLNK, S_IFREG
from sys import argv, exit, path
from time import time, sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify

sys.path.insert(0,"../tagfuse")
from tagfuse import radioutils
from tagfuse import TagFuseTree
from tagfuse import taghandlers

sys.path.insert(0,"../tagnet")
from tagnet import *

from fuse import FuseOSError

if not hasattr(__builtins__, 'bytes'):
    bytes = str

('si446x version: ', '0.2.6')
('tagnet: ', '/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '/mnt/Open/TagNet/notebooks')
tagnet driver version 0.0.25
370489632 -1220264363 16047 ('0x16153920', '-0x48bbc1ab', '0x3eaf')
-2702913 -4321127 3821828 ('-0x293e41', '-0x41ef67', '0x3a5104')
loading sparsefile
*** TagFuseTree.py ending


In [2]:
import inspect

In [3]:
from tagfuse import tagfuseargs
sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
tagfuseargs.process_cmd_args()
tagfuseargs.get_cmd_args()

('mountpoint', '/home/pi/tags')
('verbosity turned on', 1)
('sparse files stored here', '/tmp')


Namespace(background=False, disable_sparse=True, disable_sparse_read=False, mountpoint='/home/pi/tags', sparse_dir='/tmp', verbosity=1)

In [4]:
#tagfuse.tagfuseargs.global_args=None

### C structures from TAG code define image info metadata
```
typedef struct {                        /* little endian order  */
  uint16_t build;                       /* that's native for us */
  uint8_t  minor;
  uint8_t  major;
} image_ver_t;

typedef struct {
  uint8_t  hw_rev;
  uint8_t  hw_model;
} hw_ver_t;

typedef struct {
  uint32_t    sig;                      /* must be IMAGE_INFO_SIG to be valid */
  uint32_t    image_start;              /* where this binary loads            */
  uint32_t    image_length;             /* byte length of entire image        */
  uint32_t    vector_chk;               /* simple checksum over vector table  */
  uint32_t    image_chk;                /* simple checksum over entire image  */
  image_ver_t ver_id;
  hw_ver_t    hw_ver;
} image_info_t;
```

In [5]:
#def path2list(path):
#    path = os.path.abspath(os.path.realpath(path))
#    return path.split('/')[1:]

def LocateNode(tag_tree, path):
    # print(path2list(path))
    if (path == '/'):
        print('located root')
        return tag_tree
    return tag_tree.traverse(path2list(path), 0)

def DeleteNode(path, node):
    pass

def path2list(path):
    path = os.path.abspath(os.path.realpath(path))
    return path.split('/')[1:]

In [6]:
#  IMAGE_INFO provides information about a Tag software image. This data is
#  embedded in the image itself. The IMAGE_META_OFFSET is the offset into
#  the image where image_info lives in the image.  It directly follows the
#  exception vectors which are 0x140 bytes long.
# 
#  This struct will have to change, If MSP432 vector table length changes.
# 
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 0, 0, 257, 2, 3, ' '  * 40, ' '  * 40, ' '  * 30, 0xde, 2]
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IM_FIELDS = '<LLLLLHBB44s44s30sBB'
image_info_struct = pystruct.Struct(IM_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [7]:
#FILENAME    = '/tmp/test.bin'
FILENAME    = 'blink464.bin'
FILENAME    = 'main386.bin'
#FILENAME    = 'main442.bin'
#FILENAME    = 'main443.bin'

In [8]:
try:
    infile = open(FILENAME, 'rb')
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
        main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginning
    
finally:
    infile.close()

file information
  signature: 0x33275401, start: 0x20000, length: 0xb398, vect_chk: 0x0, image_chk: 0x0
  version: (0.2.386(0x182)), rev: 1, model: 240


In [9]:
def _file_size(fd):
    fd.seek(0, 2) # seek to the end
    _size = fd.tell()
    if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    fd.seek(0, 0)    # seek to the beginnnig
    return _size

In [10]:
def get_image_info(filename):
    print(filename)
    infile = open(filename, 'rb')
    file_size = _file_size(infile)

    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
            main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG:
        raise RadioLoadException("image info is corrupted")
    if imagelength != file_size:
        raise RadioLoadException("file size doesn't match image info, file: {}, info: {}".format(
                                                                                        file_size,
                                                                                        imageLength))
    infile.seek(0)    # seek back to the beginnnig
    return (infile, (im_major, im_minor, im_build), imagelength)

In [11]:
fd, vers, size = get_image_info(FILENAME)
fd.close()
print(vers,size)

main386.bin
file information
  signature: 0x33275401, start: 0x20000, length: 0xb398, vect_chk: 0x0, image_chk: 0x0
  version: (0.2.386(0x182)), rev: 1, model: 240
(0, 2, 386) 45976


In [12]:
def default_image(filename):
    # write out simple default binary input file for testing purposes
    #
    if not os.path.isfile(filename):
        with open(filename,'wb') as outfile:
            buf = bytearray(IMAGE_META_OFFSET)
            for x in range(1,IMAGE_META_OFFSET): buf[x] = x & 0x7f
            outfile.write(buf)
            outfile.write(bytearray(image_info_struct.pack(*IMAGE_INFO_DEFAULT)))
            for i in range(64):
                for x in range(256): buf[x] = x & 0x7f
                outfile.write(buf)

In [13]:
def get_name_from_class(model):
    c = model.__class__.__mro__[0]
    name = c.__module__ + "." + c.__name__
    return name

In [14]:
radio = radioutils.radio_start()

/mnt/Open/TagNet/tagfuse/../si446x/si446x/si446xradio.py:711: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(GPIO_SDN,GPIO.OUT)   #  [sdn]


In [15]:
tag_tree = TagFuseTree.TagFuseRootTree(radio)
tree_root = tag_tree

sparse handler init: /tmp


In [16]:
path='/'
handler = LocateNode(tag_tree, path)
print(type(handler))

located root
<class 'tagfuse.taghandlers.PollNetDirHandler'>


In [17]:
h = tree_root
isinstance(h, taghandlers.DirHandler)

True

In [18]:
res = inspect.getmembers(h)
for x, y in res:
    if (x == '__class__'):
        break
print(y,type(y),type(h),type(taghandlers.DirHandler))
print(isinstance(h,y), isinstance(h,taghandlers.DirHandler), isinstance(y,taghandlers.DirHandler))
print(issubclass(type(h),y), issubclass(type(h),taghandlers.DirHandler), issubclass(type(y),taghandlers.DirHandler))

<class 'tagfuse.taghandlers.PollNetDirHandler'> <type 'type'> <class 'tagfuse.taghandlers.PollNetDirHandler'> <type 'type'>
True True False
True True False


In [19]:
h['']

FileHandler([('st_ctime', 1523517058.096736),
             ('st_mtime', 1523517058.096738),
             ('st_nlink', 4),
             ('st_gid', 1000),
             ('st_blksize', 512),
             ('st_size', 0),
             ('st_atime', 1523517058.09674),
             ('st_uid', 1000),
             ('st_mode', 16873)])

In [20]:
handler['']

FileHandler([('st_ctime', 1523517058.096736),
             ('st_mtime', 1523517058.096738),
             ('st_nlink', 4),
             ('st_gid', 1000),
             ('st_blksize', 512),
             ('st_size', 0),
             ('st_atime', 1523517058.09674),
             ('st_uid', 1000),
             ('st_mode', 16873)])

In [21]:
handlerA = LocateNode(tree_root ,'/ffffffffffff/tag/sd/0/dblk')
handlerA

(DirHandler([('',
              FileHandler([('st_ctime', 1523517058.107387),
                           ('st_mtime', 1523517058.107388),
                           ('st_nlink', 7),
                           ('st_gid', 1000),
                           ('st_blksize', 512),
                           ('st_size', 0),
                           ('st_atime', 1523517058.107389),
                           ('st_uid', 1000),
                           ('st_mode', 16873)])),
             ('byte',
              SparseIOFileHandler([('st_ctime', 1523517058.107766),
                                   ('st_mtime', 1523517058.107767),
                                   ('st_nlink', 1),
                                   ('st_gid', 1000),
                                   ('st_blksize', 512),
                                   ('st_size', 0),
                                   ('st_atime', 1523517058.107768),
                                   ('st_uid', 1000),
                                   (

In [22]:
handlerB = tree_root['ffffffffffff']['tag']['sd']['0']['dblk']
handlerB

DirHandler([('',
             FileHandler([('st_ctime', 1523517058.107387),
                          ('st_mtime', 1523517058.107388),
                          ('st_nlink', 7),
                          ('st_gid', 1000),
                          ('st_blksize', 512),
                          ('st_size', 0),
                          ('st_atime', 1523517058.107389),
                          ('st_uid', 1000),
                          ('st_mode', 16873)])),
            ('byte',
             SparseIOFileHandler([('st_ctime', 1523517058.107766),
                                  ('st_mtime', 1523517058.107767),
                                  ('st_nlink', 1),
                                  ('st_gid', 1000),
                                  ('st_blksize', 512),
                                  ('st_size', 0),
                                  ('st_atime', 1523517058.107768),
                                  ('st_uid', 1000),
                                  ('st_mode', 33060)]))

In [23]:
print(handlerA, handlerB)
handlerA == handlerB

(DirHandler([('', FileHandler([('st_ctime', 1523517058.107387), ('st_mtime', 1523517058.107388), ('st_nlink', 7), ('st_gid', 1000), ('st_blksize', 512), ('st_size', 0), ('st_atime', 1523517058.107389), ('st_uid', 1000), ('st_mode', 16873)])), ('byte', SparseIOFileHandler([('st_ctime', 1523517058.107766), ('st_mtime', 1523517058.107767), ('st_nlink', 1), ('st_gid', 1000), ('st_blksize', 512), ('st_size', 0), ('st_atime', 1523517058.107768), ('st_uid', 1000), ('st_mode', 33060)])), ('note', SimpleRecHandler([('st_ctime', 1523517058.109622), ('st_mtime', 1523517058.109624), ('st_nlink', 1), ('st_gid', 1000), ('st_blksize', 512), ('st_size', 0), ('st_atime', 1523517058.109624), ('st_uid', 1000), ('st_mode', 33200)])), ('.recnum', ByteIOFileHandler([('st_ctime', 1523517058.110219), ('st_mtime', 1523517058.11022), ('st_nlink', 1), ('st_gid', 1000), ('st_blksize', 512), ('st_size', 0), ('st_atime', 1523517058.110221), ('st_uid', 1000), ('st_mode', 33060)])), ('.last_rec', ByteIOFileHandler([(

False

In [24]:
tree_root['ffffffffffff']['tag']['sd']['0']['dblk']['byte']

SparseIOFileHandler([('st_ctime', 1523517058.107766),
                     ('st_mtime', 1523517058.107767),
                     ('st_nlink', 1),
                     ('st_gid', 1000),
                     ('st_blksize', 512),
                     ('st_size', 0),
                     ('st_atime', 1523517058.107768),
                     ('st_uid', 1000),
                     ('st_mode', 33060)])

In [25]:
if isinstance(handlerA, taghandlers.DirHandler): print('ok')

In [26]:
path='/ffffffffffff'
handler, path_list = LocateNode(tag_tree, path)
type(handler), path_list

(tagfuse.taghandlers.DirHandler, ['<node_id:ffffffffffff>'])

In [27]:
handler.getattr(path2list(path), update=True)

*** getattr ['ffffffffffff']


FileHandler([('st_ctime', 1523517058.1018),
             ('st_mtime', 1523517058.101801),
             ('st_nlink', 3),
             ('st_gid', 1000),
             ('st_blksize', 512),
             ('st_size', 0),
             ('st_atime', 1523517058.101802),
             ('st_uid', 1000),
             ('st_mode', 16873)])

In [28]:
path='/ffffffffffff/tag/sd'
handler, path_list = LocateNode(tag_tree, path)
type(handler), path_list

(tagfuse.taghandlers.DirHandler, ['<node_id:ffffffffffff>', 'tag', 'sd'])

## Poll for Tags

In [29]:
STOP

In [ ]:
tagpath='/'

In [53]:
handler, path_list = LocateNode(tag_tree, tagpath)
type(handler), path_list

(tagfuse.taghandlers.DirHandler, ['<node_id:ffffffffffff>'])

In [50]:
img_list = handler.readdir(path_list)

im get dir, state: v version: (0, 2, 386)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 2, 386) valid
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set(['0.2.386'])
my_set Set([])


## Examine Image Directory

In [29]:
STOP

In [30]:
tagpath='/ffffffffffff/tag/sd/0/img'

In [31]:
handler, path_list = LocateNode(tag_tree, tagpath)
type(handler), path_list

(tagfuse.taghandlers.ImageDirHandler,
 ['<node_id:ffffffffffff>', 'tag', 'sd', '0', 'img'])

In [50]:
img_list = handler.readdir(path_list)

im get dir, state: v version: (0, 2, 386)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 2, 386) valid
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set(['0.2.386'])
my_set Set([])


In [51]:
for a in img_list:
    print(a)

.
..
0.2.386


In [34]:
handler.radio

In [35]:
for i in range(5000):
    img_list = handler.readdir(path_list)

im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
ta

im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
ta

im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
ta

im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
ta

im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
tag_set Set([])
my_set Set([])
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
im get dir, state: x version: (0, 0, 0)
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
img readdir version/state (0, 0, 0) x
ta

KeyboardInterrupt: 

## Write Image using TagFuse API

In [36]:
def write_file(fd, handler, path_list, fsize):
    while (fsize - fd.tell() > 0):
        fpos = fd.tell()
        size = handler.write(path_list, fd.read(4096), fpos)

In [37]:
def _file_size(fd):
    fd.seek(0, 2) # seek to the end
    _size = fd.tell()
    if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    fd.seek(0, 0)    # seek to the beginnnig
    return _size

In [38]:
fd, vers, size = get_image_info(FILENAME)
(vers, size)

main386.bin
file information
  signature: 0x33275401, start: 0x20000, length: 0xb398, vect_chk: 0x0, image_chk: 0x0
  version: (0.2.386(0x182)), rev: 1, model: 240


((0, 2, 386), 45976)

In [39]:
dirpath   = '/ffffffffffff/tag/sd/0/img'
filepath  = dirpath + '/' + '.'.join(map(str,vers))
(dirpath, filepath)

('/ffffffffffff/tag/sd/0/img', '/ffffffffffff/tag/sd/0/img/0.2.386')

In [40]:
STOP

NameError: name 'STOP' is not defined

In [41]:
filehandler, path_list = LocateNode(tag_tree, filepath)
type(filehandler), path_list

*** dirhandler.traverse fail


(NoneType, None)

In [ ]:
dirhandler, _ = LocateNode(tag_tree, dirpath)
type(dirhandler)

In [ ]:
mode = 0o0666
dirhandler.create(path_list, mode)

In [ ]:
dirhandler

In [ ]:
fd.tell()
write_file(fd, filehandler, path_list, size)

In [ ]:
filehandler.flush(path_list)

In [ ]:
attrs = filehandler.getattr(path_list, update=True)
attrs

In [ ]:
filehandler.flush(path_list)

## Load Image File onto Tag using radioutils

In [ ]:
STOP

In [42]:
version='0.2.442'
dirpath='/ffffffffffff/tag/sd/0/img'
filepath=dirpath+'/'+version

In [43]:
dirhandler, path_list = LocateNode(tag_tree, dirpath)
(type(dirhandler), path_list)

(tagfuse.taghandlers.ImageDirHandler,
 ['<node_id:ffffffffffff>', 'tag', 'sd', '0', 'img'])

In [ ]:
STOP

In [44]:
radio.trace._enable()
# set rssi control (enable latch, average4, latch on sync)
prp         = bytearray('0x22')
radio.set_property('MODEM', 0x4c, prp) 
# tx/rx threshold
#prp = bytearray('\x20\x20')
#radio.set_property('PKT', 0x0b, prp)

In [45]:
# default paramters
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
SHORT_DELAY         = 0

In [46]:
class RadioLoadException(Exception):
    pass

In [47]:
def upload_image(radio, tagpath, filename):
    
    def _file_size(fd):
        fd.seek(0, 2) # seek to the end
        _size = fd.tell()
        if _size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
        fd.seek(0, 0)    # seek to the beginnnig
        return _size

    def _put_msg(fd, path, version, file_size, eod=False):
        tlv_list = radioutils.path2tlvs(path_list)
        tlv_list.append(TagTlv(tlv_types.VERSION, version))
        if (fd.tell() > 0):
            tlv_list.append(TagTlv(tlv_types.OFFSET, fd.tell()))
        # build the PUT mesage object
        req_obj = TagPut(TagName(tlv_list))
        # optionally add payload
        if eod:
            # send end of data indication
            pload = TagTlvList([TagTlv(tlv_types.EOF)])
        elif (fd.tell() < file_size):
            # determine payload size to send and read it
            chunk_size = MAX_PAYLOAD - req_obj.pkt_len()
            if ((file_size - fd.tell()) < chunk_size):
                chunk_size = file_size - fd.tell()
            pload = bytearray(fd.read(chunk_size))
        # else
            # just send without payload
        if (pload is not None):
            req_obj.payload = pload
        return req_obj, len(pload)

    start = datetime.now()
    print(start)
    try:
        # open input file and determine its length
        print(filename)
        infile = open(filename, 'rb')
        file_size = _file_size(infile)

        # get image info from input file and sanity check
        infile.seek(IMAGE_META_OFFSET) # seek to location of image info
        image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
        print("file information")
        sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
            main_tree, aux_tree, build_time, im_rev, im_model = image_info
        pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
        print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
        pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
        print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
        if sig != IMAGE_INFO_SIG:
            raise RadioLoadException("image info is corrupted")
        if imagelength != file_size:
            raise RadioLoadException("file size doesn't match image info, file: {}, info: {}".format(
                                                                                        file_size,
                                                                                        imageLength))
        infile.seek(0)    # seek back to the beginnnig

        # loop to transfer image data to tag
        retries         = 0
        total_retries   = 0
        packets_sent    = 0
        while (file_size - infile.tell() > 0):
            if ((packets_sent % 10) == 0):
                print("\r{0:.2f}%".format((float(infile.tell())/float(file_size))*100), end='')
            #    print(">>>> file size: {}, offset: {}, retries: {}, rssi: {}\r".format(file_size, infile.tell(), retries, rssi, end=''))
            req_msg, plen = _put_msg(infile, tagpath, (im_major, im_minor, im_build), file_size)
            err, payload = radioutils.msg_exchange(radio, req_msg)
            packets_sent = packets_sent + 1
            if (err) and (err == tlv_errors.EALREADY):
                raise RadioLoadException('file already exists')
            offset = None
            if (err is None) or (err == tlv_errors.SUCCESS):
                offset = radioutils.payload2values(payload,
                                     [tlv_types.OFFSET,
                                     ])[0]
                # zzz print(offset, err)
            if offset and offset != infile.tell():
                if (err): p = "{}".format(err)
                else:     p = "BAD OFFSET"
                print("\n{}  {}! offset: response {}, expected {}".format(
                                                                        datetime.now(),
                                                                        p,
                                                                        offset,
                                                                        infile.tell()))
                if (offset >= 0):
                    infile.seek(offset, 0)
                else:
                    infile.seek(-plen, 1)
                    offset -= plen
                retries = retries + 1
                total_retries = total_retries + 1
            else:
                retries = 0
            if (retries > MAX_RETRIES):
                raise RadioLoadException("too many retries")
            sleep(SHORT_DELAY)

        # send end of file to complete the image load operation
        
        req_msg, plen = _put_msg(infile, tagpath, (im_major, im_minor, im_build), file_size, eod=True)
        err, payload = radioutils.msg_exchange(radio, req_msg)
        if (err is None) or (err == tlv_errors.SUCCESS):
            offset = radioutils.payload2values(payload,
                                     [tlv_types.OFFSET,
                                     ])[0]
#            if (offset is none): offset = -1
#        else:
#            offset = -1
        print("\ntotals bytes: {}, packets: {} retries: {}".format(offset,
                                                                   packets_sent,
                                                                   total_retries))
    except RadioLoadException as e:
        print('terminating:', e)
    finally:
        infile.close()
    print('\ndone, elapsed time', datetime.now() - start)

In [48]:
TAG_PATH = '/ffffffffffff/tag/sd/0/img'

In [49]:
upload_image(radio, TAG_PATH, FILENAME)

2018-04-12 00:12:29.199299
main386.bin
file information
  signature: 0x33275401, start: 0x20000, length: 0xb398, vect_chk: 0x0, image_chk: 0x0
  version: (0.2.386(0x182)), rev: 1, model: 240
65.48%radio send command error
radio receive cmd error
msg_exchange: timeout
msg_exchange: timeout
98.21%
totals bytes: 45976, packets: 214 retries: 0

done, elapsed time 0:01:30.739108


## Get Version of 'active', 'standby', 'golden', 'nib', and 'running' Images

In [ ]:
STOP

In [ ]:
def get_version(radio, path, which):
    
    def _get_version_msg(path, which):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.append(TagTlv(tlv_types.STRING, which))
        req_obj = TagGet(TagName(tlv_list))
        return req_obj

    req_msg = _get_version_msg(path, which)
    # zzz print(req_msg.name)
    err, payload = msg_exchange(radio, req_msg)
    # zzz print(payload)
    if (err == None):
        err = tlv_errors.SUCCESS
    if err == tlv_errors.SUCCESS:
        version, state = payload2values(payload,
                                [tlv_types.VERSION,
                                 tlv_types.STRING,
                                ])
    else:
        version = (0,0,0)
        state = 'x'

    return err, version, state

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
print(get_version(radio, TAG_PATH, 'active'))
print(get_version(radio, TAG_PATH, 'backup'))
print(get_version(radio, TAG_PATH, 'running'))
print(get_version(radio, TAG_PATH, 'nib',))
print(get_version(radio, TAG_PATH, 'golden'))

## Delete Version Image

In [ ]:
STOP

In [ ]:
def delete_image(radio, path, version):
    
    def _delete_msg(path, version):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.append(TagTlv(tlv_types.VERSION, version))
        req_obj = TagDelete(TagName(tlv_list))
        return req_obj

    req_msg = _delete_msg(path, version)
    print(req_msg.name)
    err, payload = msg_exchange(radio, req_msg)
    print(payload)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sd/0/img'

In [ ]:
print(delete_image(radio, TAG_PATH, (0,2,386)))

## Set Version of 'active' and 'backup' Images

In [ ]:
STOP

In [ ]:
def set_version(radio, path, which, version):
    
    def _set_version_msg(path, which, version):
        tlv_list = path2tlvs(path2list(path))
        tlv_list.extend([
            TagTlv(tlv_types.STRING, which),
            TagTlv(tlv_types.VERSION, version),
           ])
        req_obj = TagPut(TagName(tlv_list))
        return req_obj

    req_msg = _set_version_msg(path, which, version)
    err, payload = msg_exchange(radio, req_msg)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
print(set_version(radio, TAG_PATH, 'active', (0, 2, 384)))
#print(set_version(radio, TAG_PATH, 'backup', (0, 2, 386)))

## Get Poll Count and Events

In [ ]:
STOP

In [ ]:
def get_poll_info(radio, path, name):
    
    def _get_poll_msg(path, name):
        tlv_list = radioutils.path2tlvs(path2list(path))
        tlv_list.append(TagTlv(name))
        req_obj = TagHead(TagName(tlv_list))
        return req_obj

    req_msg = _get_poll_msg(path, name)
    err, payload = radioutils.msg_exchange(radio, req_msg)
    if (err is None) or (err == tlv_errors.SUCCESS):
        size = radioutils.payload2values(payload,
                                [tlv_types.SIZE,
                                ])[0]

        return size
    return None

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/poll'

In [ ]:
print('poll count', get_poll_info(radio, TAG_PATH, 'cnt'))
print('poll events', get_poll_info(radio, TAG_PATH, 'ev'))

## Reboot into 'active', 'standby', 'golden', 'nib', and 'running' Images

In [ ]:
STOP

In [ ]:
def reboot_version(radio, path, which, version):
    
    def _reboot_version_msg(path, which, version):
        tlv_list = radioutils.path2tlvs(radioutils.path2list(path))
        tlv_list.extend([
            TagTlv(tlv_types.STRING, which),
            TagTlv(tlv_types.VERSION, version),
           ])
        req_obj = TagPut(TagName(tlv_list))
        return req_obj

    req_msg = _reboot_version_msg(path, which, version)
    err, payload = radioutils.msg_exchange(radio, req_msg)
    if (err is None):
        err = tlv_errors.SUCCESS
    return err

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
reboot_version(radio, TAG_PATH, 'active', (0, 1, 0))
#set_version(radio, TAG_PATH, 'backup', (0, 1, 0))
#set_version(radio, TAG_PATH, 'running', (0, 1, 0))
#set_version(radio, TAG_PATH, 'nib', (0, 1, 0))
reboot_version(radio, TAG_PATH, 'golden', (0, 1, 0))

## other Image

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

In [ ]:
STOP

In [ ]:
TAG_PATH = '/<node_id:ffffffffffff>/tag/sys'

## OTHER FUNCTIONS

In [ ]:
STOP

In [ ]:
handler.radio.trace.display(handler.radio.trace.filter())

In [ ]:
radio.get_clear_interrupts()

In [ ]:
for i in range(100):
    print(radio.get_clear_interrupts())

In [ ]:
radio.dump_radio()

# 